# 13F - New Purchases, Increases and Largest Holdings

In [116]:
import pandas as pd
import os
from datetime import datetime
import numpy as np

In [117]:
mapping1 = pd.read_csv(r"C:\Users\bfass\OneDrive\Desktop\Fin tools\BigMoney13F\Final_cusip_map\CUSIP_Mapping.csv", usecols = ["CUSIP", "SYMBOL"])
mapping2 = pd.read_csv(r"C:\Users\bfass\OneDrive\Desktop\Fin tools\BigMoney13F\Final_cusip_map\CUSIP_Mapping-Brocks-Laptop.csv", usecols = ["CUSIP", "SYMBOL"])
mapping = pd.concat([mapping1, mapping2]).drop_duplicates()

In [118]:
directory = "Holdings_shares"

# Create an empty dictionary to store DataFrames
dfs = {}

# Iterate through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        # Construct the full file path
        filepath = os.path.join(directory, filename)
        
        # Read the CSV file into a DataFrame
        df_name = os.path.splitext(filename)[0]  # Get the file name without the extension
        
        read_df = pd.read_csv(filepath)
        
        read_df = read_df.merge(mapping[["CUSIP", "SYMBOL"]].drop_duplicates(), how = 'left', on = "CUSIP")

        tbl = read_df.groupby("SYMBOL").max().reset_index()
        
        dfs[df_name] = tbl
        #print(df_name)

C:\Users\bfass\AppData\Local\Temp\ipykernel_27064\4015541582.py:19: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  tbl = read_df.groupby("SYMBOL").max().reset_index()
C:\Users\bfass\AppData\Local\Temp\ipykernel_27064\4015541582.py:19: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  tbl = read_df.groupby("SYMBOL").max().reset_index()
C:\Users\bfass\AppData\Local\Temp\ipykernel_27064\4015541582.py:19: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  tbl = read_df.groupby("SYMBOL").max().reset_index()
C:\User

In [119]:
pos_chg_df = pd.DataFrame(columns=["SYMBOL", "Position_Change", "Fund"])


for fund in dfs.keys():
    cols = dfs[fund].columns
    cols_filt = dfs[fund].columns.str.contains("-")

    rec_qs = cols[cols_filt][0:2]

    fund_df = dfs[fund][["SYMBOL"] + list(rec_qs)].fillna(1)

    fund_df = fund_df[fund_df["SYMBOL"] != 1]

    fund_df['Share_Delta'] = fund_df[rec_qs[0]] - fund_df[rec_qs[1]]

    fund_df["Inc_Position"] = np.where(fund_df['Share_Delta'] > 0, 1, 0)
    fund_df["Decr_Position"] = np.where(fund_df['Share_Delta'] < 0, 1, 0)
    fund_df["Decr_Position_50_per"] = np.where(fund_df[rec_qs[0]]/fund_df[rec_qs[1]] < .5, 1, 0)

    fund_df["Exit_Position"] = np.where((fund_df[rec_qs[0]] == 1) & (fund_df[rec_qs[1]] > 1), 1, 0)

    fund_df["Inc_Position_50_per"] = np.where(fund_df[rec_qs[0]]/fund_df[rec_qs[1]] >= 1.5, 1, 0)
    fund_df["Inc_Position_100_per"] = np.where(fund_df[rec_qs[0]]/fund_df[rec_qs[1]] >= 2, 1, 0)
    fund_df['New_Holding'] = np.where((fund_df[rec_qs[0]] > 1) & (fund_df[rec_qs[1]] == 1), 1, 0)

    for index, row in fund_df.iterrows():
        for col in row.keys():
            if (col in ["Decr_Position","Decr_Position_50_per", "Exit_Position","New_Holding", "Inc_Position", "Inc_Position_50_per", "Inc_Position_100_per"]) and (row[col] == 1): ## Probably should create a hierarchy for readability

                new_row = {"SYMBOL": row["SYMBOL"], "Position_Change": col, "Fund": fund}

                pos_chg_df = pd.concat([pos_chg_df, pd.DataFrame([new_row])], ignore_index=True)
                



In [120]:
pos_chg_df["Hierarchy"] = np.where(pos_chg_df["Position_Change"].isin(["New_Holding","Exit_Position"]), 1,
                                    np.where(pos_chg_df["Position_Change"].isin(["Inc_Position_100_per","Decr_Position_50_per"]), 2,
                                     np.where(pos_chg_df["Position_Change"].isin(["Decr_Position","Inc_Position_50_per"]), 3, 4)))

In [134]:
adj_change = pos_chg_df.groupby(["Fund", "SYMBOL"])["Hierarchy"].min().reset_index()

pos_chg_df_uniq = pos_chg_df.merge(adj_change, on=["Fund", "SYMBOL","Hierarchy"], how="inner")

In [135]:
pos_chg_df_uniq["Score"] = np.where(pos_chg_df_uniq["Position_Change"].isin(["New_Holding","Exit_Position"]), 3,
                                    np.where(pos_chg_df_uniq["Position_Change"].isin(["Inc_Position_100_per","Decr_Position_50_per"]), 2,
                                     np.where(pos_chg_df_uniq["Position_Change"].isin(["Decr_Position","Inc_Position_50_per"]), 1, .5)))

pos_chg_df_uniq["Score"] = np.where(pos_chg_df_uniq["Position_Change"].isin(["Decr_Position","Decr_Position_50_per", "Exit_Position"]), pos_chg_df_uniq["Score"] * -1, pos_chg_df_uniq["Score"])

In [136]:
scoring = pos_chg_df_uniq.groupby("SYMBOL")["Score"].agg(func=["sum", "mean", "count"]).reset_index()
scoring.columns = ("SYMBOL","Total Score", "Mean_Score", "# Funds Scoring")

In [137]:
result = pos_chg_df_uniq.pivot_table(
    index="SYMBOL", 
    columns="Position_Change", 
    values="Fund", 
    aggfunc=lambda x: " || ".join(x)
).reset_index()

In [138]:
final_df = result[["SYMBOL", "New_Holding", "Inc_Position_100_per", "Inc_Position_50_per", "Inc_Position", "Exit_Position", "Decr_Position_50_per", "Decr_Position"]]

final_df = final_df.merge(scoring, on = "SYMBOL", how="left")

final_df.sort_values(by="# Funds Scoring", ascending=False).head(30)

final_df.sort_values(by=["Mean_Score", "# Funds Scoring"], ascending=False).head(30)

,SYMBOL,New_Holding,Inc_Position_100_per,Inc_Position_50_per,Inc_Position,Exit_Position,Decr_Position_50_per,Decr_Position,Total Score,Mean_Score,# Funds Scoring
842,SEG,Ancora || Bill_Ackman,NaN,NaN,NaN,NaN,NaN,NaN,6.0,3.0,2
912,TEVA,Stanley_Druckenmiller || VIKING GLOBAL INVESTO...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,3.0,2
2,AAOI,Whale Rock Capital Management LLC,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,1
5,ABB,Ancora,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,1
6,ABBNY,Ancora,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,1
8,ABCB,Stanley_Druckenmiller,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,1
16,ACWI,Ancora,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,1
24,AEE,Ancora,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,1
36,AKAM,Ancora,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,1
43,ALLY,"Brave Warrior Advisors, LLC",NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,1
